First, I open and explore the datasets to try to resolve any inconsistencies before beginning to analyze the data.

In [8]:
import pandas as pd
import os

In [9]:
politicians_raw = pd.read_csv("../raw_data/politicians_by_country_AUG.2024.csv")
population_raw = pd.read_csv("../raw_data/population_by_country_AUG.2024.csv")

Often, when working with geographic data, it's challenging to perform joins because regions have different names in different datasets. Let's check if that's the case here.

In [10]:
print(set(politicians_raw["country"]) - set(population_raw["Geography"]))
print(set(population_raw["Geography"]) - set(politicians_raw["country"]))

{'Korea, South', 'Korean', 'Guinea-Bissau'}
{'Palau', 'Brunei', 'China (Hong Kong SAR)', 'San Marino', 'Martinique', 'CENTRAL AMERICA', 'WESTERN EUROPE', 'OCEANIA', 'NORTHERN AMERICA', 'Nauru', 'SOUTH ASIA', 'ASIA', 'Canada', 'Philippines', 'Guam', 'MIDDLE AFRICA', 'LATIN AMERICA AND THE CARIBBEAN', 'Netherlands', 'Ireland', 'GuineaBissau', 'New Zealand', 'WESTERN AFRICA', 'Romania', 'Mexico', 'Korea (South)', 'EASTERN EUROPE', 'Puerto Rico', 'SOUTH AMERICA', 'Liechtenstein', 'New Caledonia', 'Guadeloupe', 'NORTHERN EUROPE', 'WORLD', 'WESTERN ASIA', 'Suriname', 'Korea (North)', 'Dominica', 'Mayotte', 'CENTRAL ASIA', 'Australia', 'French Guiana', 'Curacao', 'SOUTHEAST ASIA', 'eSwatini', 'United Kingdom', 'Sao Tome and Principe', 'Fiji', 'SOUTHERN AFRICA', 'United States', 'China (Macao SAR)', 'CARIBBEAN', 'Western Sahara', 'EAST ASIA', 'Andorra', 'French Polynesia', 'Kiribati', 'Jamaica', 'Iceland', 'Mauritius', 'NORTHERN AFRICA', 'Georgia', 'EUROPE', 'AFRICA', 'SOUTHERN EUROPE', 'Reuni

In [11]:
politicians_cleaned = politicians_raw.copy()
politicians_cleaned["country"] = politicians_cleaned["country"].replace(
    {
        "Korea, South": "Korea (South)",
        "Korean": "Korea (South)"
    }
)

population_cleaned = population_raw.copy()
population_cleaned["Geography"] = population_cleaned["Geography"].replace(
    {
        "GuineaBissau": "Guinea-Bissau"
    }
)

print(set(politicians_cleaned["country"]) - set(population_cleaned["Geography"]))


set()


In [12]:
out_directory = "../cleaned_data/"

if not os.path.exists(out_directory):
    os.makedirs(out_directory)
    print(f"Created '{out_directory}' folder to store cleaned data")
else:
    print(f"Despositing cleaned data in the folder: '{out_directory}'")

population_cleaned.to_csv(out_directory + "population_by_country_AUG_2024_clean.csv")
politicians_cleaned.to_csv(out_directory + "politicians_by_country_AUG_2024_clean.csv")

Despositing cleaned data in the folder: '../cleaned_data/'
